In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
tsla_df = pd.read_csv('Resources/yearly.txt')
tsla_df.head(10)

,Ticker,Year,Revenue,EPS,NetIncome,GrossProfit,OperatingIncome,EBITDA,Shares
0,TSLA,2020,31536,0.64,690,6630,1994,4496,1083
1,TSLA,2019,24578,-0.98,-870,4069,-69,2273,887
2,TSLA,2018,21461,-1.14,-976,4042,-388,1672,853
3,TSLA,2017,11759,-2.37,-1962,2223,-1632,95,830
4,TSLA,2016,7000,-0.94,-675,1599,-667,374,721
5,TSLA,2015,4046,-1.39,-889,924,-717,-216,641
6,TSLA,2014,3198,-0.47,-294,882,-187,115,623
7,TSLA,2013,2013,-0.12,-74,456,-61,60,597
8,TSLA,2012,413,-0.74,-396,30,-394,-365,537
9,TSLA,2011,204,-0.51,-254,62,-251,-235,502


In [2]:
# Generate our categorical variable list
tsla_cat = tsla_df.dtypes[tsla_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
tsla_df[tsla_cat].nunique()

Ticker    1
dtype: int64

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(tsla_df[tsla_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(tsla_cat)
encode_df.head()

,Ticker_TSLA
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [4]:
# Merge one-hot encoded features and drop the originals
tsla_df = tsla_df.merge(encode_df,left_index=True, right_index=True)
tsla_df = tsla_df.drop(tsla_cat,1)
tsla_df.head()

,Year,Revenue,EPS,NetIncome,GrossProfit,OperatingIncome,EBITDA,Shares,Ticker_TSLA
0,2020,31536,0.64,690,6630,1994,4496,1083,1.0
1,2019,24578,-0.98,-870,4069,-69,2273,887,1.0
2,2018,21461,-1.14,-976,4042,-388,1672,853,1.0
3,2017,11759,-2.37,-1962,2223,-1632,95,830,1.0
4,2016,7000,-0.94,-675,1599,-667,374,721,1.0


In [6]:
# set x and y
y = tsla_df
X = tsla_df

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

NameError: name 'X_train_scaled' is not defined

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

NameError: name 'X_train_scaled' is not defined